In [13]:
import pandas as pd
pd.set_option('display.max_columns', None)
# 读取数据并量化
df = pd.read_csv ('/Users/lycoris/Desktop/other/github/MCM2024/Data/Wimbledon_featured_matches.csv')
for i, score in enumerate(['15', '30', '40', 'AD']):
    df.loc[(df.p1_score== score) , 'p1_score'] = i + 1
    df.loc[(df.p2_score== score) , 'p2_score'] = i + 1
df[ 'p1_score' ] = df['p1_score' ].astype(int)
df[ 'p2_score' ] = df['p2_score'].astype(int)
df = df.drop(["match_id", 'player1', 'player2', "serve_width", "serve_depth","return_depth"], axis=1)
df.dropna(inplace=True)
df

,player1,player2,elapsed_time,set_no,game_no,point_no,p1_sets,p2_sets,p1_games,p2_games,p1_score,p2_score,server,serve_no,point_victor,p1_points_won,p2_points_won,game_victor,set_victor,p1_ace,p2_ace,p1_winner,p2_winner,winner_shot_type,p1_double_fault,p2_double_fault,p1_unf_err,p2_unf_err,p1_net_pt,p2_net_pt,p1_net_pt_won,p2_net_pt_won,p1_break_pt,p2_break_pt,p1_break_pt_won,p2_break_pt_won,p1_break_pt_missed,p2_break_pt_missed,p1_distance_run,p2_distance_run,rally_count,speed_mph,serve_width,serve_depth,return_depth
0,Carlos Alcaraz,Nicolas Jarry,00:00:00,1,1,1,0,0,0,0,0,0,1,2,2,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,6.000,7.840,2,95.0,BC,NCTL,ND
1,Carlos Alcaraz,Nicolas Jarry,00:00:38,1,1,2,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.253,7.094,1,118.0,B,CTL,ND
2,Carlos Alcaraz,Nicolas Jarry,00:01:01,1,1,3,0,0,0,0,1,1,1,1,2,1,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,13.800,19.808,4,120.0,B,NCTL,D
3,Carlos Alcaraz,Nicolas Jarry,00:01:31,1,1,4,0,0,0,0,1,2,1,1,1,2,2,0,0,0,0,1,0,F,0,0,0,0,0,1,0,0,0,0,0,0,0,0,51.108,75.631,13,130.0,BW,CTL,D
5,Carlos Alcaraz,Nicolas Jarry,00:02:50,1,1,6,0,0,0,0,3,2,1,2,2,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.291,4.249,2,97.0,BW,NCTL,ND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7279,Carlos Alcaraz,Novak Djokovic,04:39:58,5,10,330,2,2,5,4,0,1,1,1,1,165,165,0,0,0,0,1,0,F,0,0,0,0,0,1,0,0,0,0,0,0,0,0,38.620,50.971,11,123.0,C,NCTL,D
7280,Carlos Alcaraz,Novak Djokovic,04:40:43,5,10,331,2,2,5,4,1,1,1,1,1,166,165,0,0,0,0,1,0,B,0,0,0,0,1,0,1,0,0,0,0,0,0,0,17.533,27.766,5,114.0,W,NCTL,ND
7281,Carlos Alcaraz,Novak Djokovic,04:41:20,5,10,332,2,2,5,4,2,1,1,1,2,166,166,0,0,0,0,0,1,F,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10.882,18.328,4,124.0,W,NCTL,D
7282,Carlos Alcaraz,Novak Djokovic,04:41:52,5,10,333,2,2,5,4,2,2,1,1,1,167,166,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.893,10.537,1,130.0,C,NCTL,D


In [2]:
# 提取所需特证
x1_ls,x2_ls,x3_ls,x4_ls,x5_ls,x6_ls,x7_ls,x8_ls,x9_ls,x10_ls,\
x12_ls,x13_ls,x14_ls,x15_ls,x16_ls=[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
label_ls = []
label = 0

for match_id,set_no,game_no,point_no in zip(df.match_id, df.set_no, df.game_no, df.point_no):
    match = df[df.match_id == match_id]
    set_ = match[ match.set_no == set_no]
    game_ = set_[set_.game_no == game_no]
    point_ = game_[game_.point_no == point_no]

    # 如果x>0，则对p1有利，如果x<0，则对p2有利
    x1 = point_[ 'p1_games' ].values[0]-point_[ 'p2_games' ].values[0] # p1局领先进度
    x2 = point_[ 'p1_score' ].values[0]- point_[ 'p2_score' ].values[0] # p1小分领先进度
    x3 = 1 if point_[ 'serve_no' ].values[0] == 1 else 0 # p1是否发球
    x4 = 1 if label==1 else 0 # p1上一小分是否得到
    x5 = point_[ 'p1_sets' ].values[0] - point_[ 'p2_sets' ].values[0] # p1盘领先进度
    # p1连胜局数
    x6 = 1 if 1 in game_['p1_ace'].values else 0
    x7 = 1 if 1 in game_['p1_winner'].values else 0
    x8 = 1 if 1 in game_['p1_double_fault'].values else 0
    x9 = 1 if 1 in game_['p1_unf_err'].values else 0
    x10 = game_[ 'p1_net_pt_won' ]. sum()/game_[ 'p1_net_pt' ].sum() if game_[ 'p1_net_pt' ].sum() != 0 else 0

    index = match.index.tolist( ).index(point_.index.tolist()[0])
    x12 = match.iloc [ : index+1][ 'p1_distance_run' ].sum()
    x13 = match.iloc [ index-2 :index+1][ 'p1_distance_run' ].sum()
    x14 = point_[ 'p1_distance_run' ].values[0]
    x15 = point_[ 'speed_mph' ].values[0]
    x16 = x15 * x3

    label = 1 if point_[ 'point_victor' ].values[0] == 1 else 0
    label_ls.append(label)
    x1_ls.append(x1)
    x2_ls.append(x2)
    x3_ls.append(x3)
    x4_ls.append(x4)
    x5_ls.append(x5)
    x6_ls.append(x6)
    x7_ls.append(x7)
    x8_ls.append(x8)
    x9_ls.append(x9)
    x10_ls.append(x10)
    x12_ls.append(x12)
    x13_ls.append(x13)
    x14_ls.append(x14)
    x15_ls.append(x15)
    x16_ls.append(x16)


dataset = pd.DataFrame({'x1': x1_ls, 'x2': x2_ls, 'x3': x3_ls, 'x4': x4_ls, 'x5': x5_ls, 'x6': x6_ls, 'x7': x7_ls, 'x8': x8_ls, 'x9': x9_ls, 'x10': x10_ls, 'x12': x12_ls, 'x13': x13_ls, 'x14': x14_ls, 'x15': x15_ls, 'x16': x16_ls,
                       'label': label_ls})
dataset.to_csv('/Users/lycoris/Desktop/other/github/MCM2024/Data/Dataset.csv')

In [3]:
# 归一化数据
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cols = dataset.columns[:-1]
scaler.fit(dataset[cols].values)
dataset[cols] = scaler.transform(dataset[cols].values)
dataset.to_csv('/Users/lycoris/Desktop/other/github/MCM2024/Data/ScaledDataset.csv')
dataset

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x14,x15,x16,label
0,0.555556,0.5,0.0,0.0,0.5,1.0,1.0,0.0,1.0,1.0,0.000499,0.000000,0.032636,0.333333,0.000000,0
1,0.555556,0.4,1.0,0.0,0.5,1.0,1.0,0.0,1.0,1.0,0.001334,0.000000,0.027573,0.666667,0.836879,1
2,0.555556,0.5,1.0,1.0,0.5,1.0,1.0,0.0,1.0,1.0,0.003525,0.129442,0.085503,0.695652,0.851064,0
3,0.555556,0.4,1.0,0.0,0.5,1.0,1.0,0.0,1.0,1.0,0.011639,0.362503,0.338374,0.840580,0.921986,1
4,0.555556,0.6,0.0,1.0,0.5,1.0,1.0,0.0,1.0,1.0,0.012479,0.362699,0.027830,0.362319,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5458,0.666667,0.4,1.0,0.0,0.5,0.0,1.0,0.0,1.0,1.0,0.991408,0.255970,0.253731,0.739130,0.872340,1
5459,0.666667,0.5,1.0,1.0,0.5,0.0,1.0,0.0,1.0,1.0,0.994191,0.325117,0.110805,0.608696,0.808511,1
5460,0.666667,0.6,1.0,1.0,0.5,0.0,1.0,0.0,1.0,1.0,0.995919,0.346352,0.065725,0.753623,0.879433,0
5461,0.666667,0.5,1.0,0.0,0.5,0.0,1.0,0.0,1.0,1.0,0.997172,0.187594,0.045466,0.840580,0.921986,1


* 划分数据集

In [4]:
# 划分并保存数据
training_DS_size = int(.8 * len(dataset))
training_Dataset = dataset[:training_DS_size]
training_Dataset.to_csv('/Users/lycoris/Desktop/other/github/MCM2024/Data/trainingDataset.csv')
testing_Dataset = dataset[training_DS_size:]
testing_Dataset.to_csv('/Users/lycoris/Desktop/other/github/MCM2024/Data/testingDataset.csv')

* 建立模型

In [5]:
import numpy as np
import torch.utils.data as Data
from torchvision import transforms
from torch import nn
# train_loader = Data.DataLoader(dataset=training_Dataset, batch_size=64)
# test_loader = Data.DataLoader(dataset=testing_Dataset, batch_size=64, shuffle=True, num_workers=2)

In [6]:
# 模型定义
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=15, hidden_size=16, num_layers=1, batch_first=True)
        self.fc = nn.Linear(16, 2)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [7]:
import torch
from torch.utils.data import DataLoader
import torch.utils.data as Data
# 提取特征和标签
features = dataset.iloc[:, :-1].to_numpy()
labels = dataset.iloc[:, -1].to_numpy()

# 将标签转换为 one-hot 编码
labels = torch.nn.functional.one_hot(torch.from_numpy(labels), num_classes=2)
print(labels.shape)
labels = labels[:, 0]
print(labels.shape)

# 数据分割
size = (int(0.6 * len(labels)), int(0.2 * len(labels)), len(labels) - int(0.6 * len(labels)) -  int(0.2 * len(labels)))
train_data_label, val_data_label, test_data_label = labels.split(size)
train_data_features, val_data_features, test_data_features = torch.from_numpy(features).split(size)

# 建立dataloader
train_xt = train_data_features.float()
train_yt = train_data_label.float()
train_data = Data.TensorDataset(train_xt, train_yt)
train_dataloader = Data.DataLoader(dataset=train_data ,batch_size=4, shuffle=True, num_workers=1, drop_last=False)

val_xt = val_data_features.float()
val_yt = val_data_label.float()
val_data = Data.TensorDataset(val_xt, val_yt)
val_dataloader = Data.DataLoader(dataset=val_data ,batch_size=4, shuffle=True, num_workers=1, drop_last=False)

test_xt = test_data_features.float()
test_yt = test_data_label.float()
test_data = Data.TensorDataset(test_xt, test_yt)
test_dataloader = Data.DataLoader(dataset=test_data ,batch_size=4, shuffle=True, num_workers=1, drop_last=False)

# 建立模型
model = LSTMModel()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

torch.Size([5463, 2])
torch.Size([5463])


In [8]:
print(train_data_features.shape)
print(train_xt.shape)
print(train_yt.shape)
print(train_data)
model


torch.Size([3277, 15])
torch.Size([3277, 15])
torch.Size([3277])


LSTMModel(
  (lstm): LSTM(15, 16, batch_first=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)

In [11]:
import tqdm
# 模型训练
for epoch in range(100):
    # 训练
    model.train()
    for batch_x, batch_y in tqdm.tqdm(train_dataloader):
        outputs = model(batch_x.float().unsqueeze(1))
        loss = criterion(outputs, batch_y.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 验证
    model.eval()
    with torch.no_grad():
        for batch_x, batch_y in tqdm.tqdm(val_dataloader):
            outputs = model(batch_x.float().unsqueeze(1))
            loss = criterion(outputs, batch_y.long())
            accuracy = (outputs.argmax(dim=1) == batch_y).float().mean()

    print(f"Epoch: {epoch}, Loss: {loss}, Accuracy: {accuracy}")

# 模型评估
model.eval()
with torch.no_grad():
    for batch_x, batch_y in tqdm.tqdm(test_dataloader):
        outputs = model(batch_x.float().unsqueeze(1))
        loss = criterion(outputs, batch_y.long())
        accuracy = (outputs.argmax(dim=1) == batch_y).float().mean()


100%|██████████| 273/273 [00:01<00:00, 136.78it/s]


Epoch: 0, Loss: 0.5156656503677368, Accuracy: 0.5


100%|██████████| 273/273 [00:01<00:00, 139.60it/s]


Epoch: 1, Loss: 0.4881666302680969, Accuracy: 1.0


100%|██████████| 273/273 [00:02<00:00, 133.30it/s]


...

100%|██████████| 273/273 [00:01<00:00, 138.85it/s]


Epoch: 98, Loss: 0.6439730525016785, Accuracy: 0.5


100%|██████████| 273/273 [00:01<00:00, 138.35it/s]


Epoch: 99, Loss: 0.5126151442527771, Accuracy: 0.75


100%|██████████| 274/274 [00:01<00:00, 139.12it/s]
